# IMPORTS E CONFIGURAÇÕES

## Imports

In [18]:
import sys
import pickle
import pandas as pd
import numpy as np
import requests

from io import StringIO
from flask import Flask, request, Response


# Adiciona o diretório raiz do projeto ao path do Python
sys.path.append('../')
from api.rossmann.Rossmann import Rossmann


# LOAD DATA

In [ ]:
# --- Carregar os dados brutos que o modelo nunca viu ---
df_test = pd.read_csv('../data/raw/test.csv', low_memory=False)
df_store = pd.read_csv('../data/raw/store.csv', low_memory=False)
df_raw = pd.merge(df_test, df_store, on='Store', how='left')

# DEPLOY

## Classe Rossmann

In [ ]:
# 1. Instanciar a classe Rossmann
# A classe agora carrega o modelo automaticamente no __init__
pipeline = Rossmann()

# 2. Criar uma amostra para o teste
df_sample = df_raw.sample(n=5, random_state=42)

### Realizar pré-processamento

In [ ]:
# 3. Aplicar o pipeline de pré-processamento
df_prepared = pipeline.preprocess(df_sample.copy())

### Executar o modelo de previsão de vendas

In [ ]:
# 4. Fazer a previsão (não precisa mais passar o modelo como argumento)
df_response_json = pipeline.get_prediction(df_sample, df_prepared)

### Ver resultado das previsões de vendas

In [31]:
# 5. Converter o JSON de resposta para um DataFrame para visualização
#    2. Usar StringIO para evitar o FutureWarning
df_response = pd.read_json(StringIO(df_response_json))

# 6. Visualizar o resultado com a coluna 'Prediction'
print("Amostra das previsões de vendas:")
display(df_response[['Store', 'Prediction']].head())

Amostra das previsões de vendas:


,Store,Prediction
0,69,7633.622559
1,468,5944.266113
2,728,5752.409668
3,426,5452.373047
4,275,3938.385254


## Chamada da API

> Execute o arquivo handler.py, através do comando abaixo, pelo terminal para ativar o serviço e poder testar a API localmente.
>
> ```python handler.py ```

### Definir variáveis na API

In [ ]:
# URL de desenvolvimento
url = 'http://0.0.0.0:8080/rossmann/predict'

# URL de produção
# url = 'https://rossmann-sales-forecast-flnh.onrender.com'

header = {
    'Content-type': 
    'application/json' 
}
data = df_response_json

### Executar a API

In [ ]:
r = requests.post(
    url, 
    data=data, 
    headers=header
)
print('Status Code {}'.format(r.status_code))

### Visualizar retorno da API

In [36]:
d1 = pd.DataFrame(
    r.json(), 
    columns=r.json()[0].keys()
)

d2 = (
    d1[['Store', 'Prediction']]
    .groupby('Store')
    .sum()
    .reset_index()
)

for i in range(len( d2 )):
    print(
        'A loja {} venderá R$ {:,.2f} nas próximas 6 semanas'.format(
        d2.loc[i, 'Store'],
        d2.loc[i, 'Prediction']) 
    )

A loja 69 venderá R$ 7,633.62 nas próximas 6 semanas
A loja 275 venderá R$ 3,938.39 nas próximas 6 semanas
A loja 426 venderá R$ 5,452.37 nas próximas 6 semanas
A loja 468 venderá R$ 5,944.27 nas próximas 6 semanas
A loja 728 venderá R$ 5,752.41 nas próximas 6 semanas
